## PRACTICA OBLIGATORIA: SQL

* La práctica obligatoria de esta unidad consiste en jugar con varias tablas de una base de datos y practicar no solo con ellas sino con el método merge de Pandas. Descarga este notebook en tu ordenador y trabaja en local. Ten en cuenta que tendrás que descar los directorios de imágenes y datos adicionales, si los hubiera.
* Recuerda que debes subirla a tu repositorio personal antes de la sesión en vivo para que puntúe adecuadamente.
* Recuerda también que no es necesario que esté perfecta, sólo es necesario que se vea el esfuerzo.
* Esta práctica se resolverá en la sesión en vivo correspondiente y la solución se publicará en el repo del curso.

### Ejercicio 0
Importa los paquetes y módulos que necesites a lo largo del notebook

In [1]:
import pandas as pd
import sqlite3

Si quieres, crea una función que te simplifique la ejecución de queries, usa la que has visto en los workouts o bien una que esté basada en pd.read_sql

In [2]:
ruta = "C:/Users/david/Downloads/chinook.db"
connection = sqlite3.connect(ruta)
cursor_challenge = connection.cursor()

In [3]:
# Con esta función Leemos Los datos y lo pasamos a un DataFrame de Pandas  

def sql_query(query):
    # Ejecuta la query  
    cursor_challenge.execute(query) # Recuerda que sólo funcionará si has llamado cursor_  
        # a tu cursor.

    # Almacena Los datos de la query  
    ans = cursor_challenge.fetchall()

    # Obtenemos los nombres de las columnas de la tabla  
    names = [description[0] for description in cursor_challenge.description]

    return pd.DataFrame(ans, columns=names)

### 1 QUERIES
Queremos realizar algunos análisis sobre la base de datos anterior. Contesta a las siguientes preguntas de dos formas, una empleando SQL, otra descargandote la tabla que sea necesario en un DataFrame y resolviéndola usando pandas sobre ese DataFrame

##### 1.1 Muestra los clientes de Canadá y de Brasil. Utiliza solo el nombre (completo), su identificador y su país.

In [4]:
## SQL
tablas = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", connection)
print("Tablas en la base de datos:")
print(tablas)

Tablas en la base de datos:
               name
0            albums
1   sqlite_sequence
2           artists
3         customers
4         employees
5            genres
6          invoices
7     invoice_items
8       media_types
9         playlists
10   playlist_track
11           tracks
12     sqlite_stat1
13            films


In [5]:
##SQL
query = """
SELECT 
CustomerId AS Identificador,
FirstName || ' ' || LastName AS Nombre_Completo,
Country AS Pais
FROM 
customers
WHERE 
Country IN ('Canada', 'Brazil')
ORDER BY 
Country, LastName;
"""

In [6]:
sql_query(query)

,Identificador,Nombre_Completo,Pais
0,12,Roberto Almeida,Brazil
1,1,Luís Gonçalves,Brazil
2,10,Eduardo Martins,Brazil
3,13,Fernanda Ramos,Brazil
4,11,Alexandre Rocha,Brazil
5,29,Robert Brown,Canada
6,30,Edward Francis,Canada
7,32,Aaron Mitchell,Canada
8,15,Jennifer Peterson,Canada
9,14,Mark Philips,Canada


In [7]:
###Pandas
clientes = pd.read_sql("SELECT * FROM customers", connection)

paises_seleccionados = ['Canada', 'Brazil']
clientes_filtrados = clientes[clientes['Country'].isin(paises_seleccionados)]

paises_seleccionados = ['Canada', 'Brazil']
clientes_filtrados = clientes.loc[clientes['Country'].isin(paises_seleccionados)].copy()

clientes_filtrados.loc[:, 'Nombre_Completo'] = (
    clientes_filtrados['FirstName'] + ' ' + clientes_filtrados['LastName'])
resultado_final = clientes_filtrados.loc[:, ['CustomerId', 'Nombre_Completo', 'Country']].rename(
    columns={
        'CustomerId': 'ID Cliente',
        'Country': 'País'
    }
)
resultado_final = resultado_final.sort_values(by=['País', 'Nombre_Completo'])
print (resultado_final)

    ID Cliente    Nombre_Completo    País
10          11    Alexandre Rocha  Brazil
9           10    Eduardo Martins  Brazil
12          13     Fernanda Ramos  Brazil
0            1     Luís Gonçalves  Brazil
11          12    Roberto Almeida  Brazil
31          32     Aaron Mitchell  Canada
29          30     Edward Francis  Canada
32          33     Ellie Sullivan  Canada
2            3  François Tremblay  Canada
14          15  Jennifer Peterson  Canada
13          14       Mark Philips  Canada
30          31        Martha Silk  Canada
28          29       Robert Brown  Canada


##### 1.2 Muestra una lista con los paises a los que se ha facturado, la lista no debe contener paises repetidos. Recuerda que las facturas están en la tabla invoices

In [8]:
##SQL
query = """
SELECT DISTINCT BillingCountry AS 'Países Facturados' 
FROM invoices 
ORDER BY BillingCountry
"""

In [9]:
sql_query(query)

,Países Facturados
0,Argentina
1,Australia
2,Austria
3,Belgium
4,Brazil
5,Canada
6,Chile
7,Czech Republic
8,Denmark
9,Finland


In [10]:
###Pandas
paises_facturados = pd.read_sql("SELECT BillingCountry FROM invoices", connection)

lista_paises = paises_facturados['BillingCountry'].unique()
lista_paises_ordenada = sorted(lista_paises)

resultado = pd.DataFrame(lista_paises_ordenada, columns=['Países Facturados'])

print("Países únicos a los que se ha facturado:")
print(resultado)

Países únicos a los que se ha facturado:
   Países Facturados
0          Argentina
1          Australia
2            Austria
3            Belgium
4             Brazil
5             Canada
6              Chile
7     Czech Republic
8            Denmark
9            Finland
10            France
11           Germany
12           Hungary
13             India
14           Ireland
15             Italy
16       Netherlands
17            Norway
18            Poland
19          Portugal
20             Spain
21            Sweden
22               USA
23    United Kingdom


##### 1.3 Muestra cuantos clientes de Estados Unidos hay por estado de Estados Unidos. El resultado debe estar ordenado de mayor a menor. (Nota: En el caso de SQL tendrás que usar el indicador de campo, es decir si es el campo 1,2,3 o n, para la parte de ordenación)

In [11]:
###SQL
query= '''
SELECT 
State AS Estado,
COUNT(*) AS Cantidad_Clientes
FROM 
customers
WHERE 
Country = 'USA'
AND State IS NOT NULL
GROUP BY 
State
ORDER BY 
COUNT(*) DESC;
'''

In [12]:
sql_query(query)

,Estado,Cantidad_Clientes
0,CA,3
1,WI,1
2,WA,1
3,UT,1
4,TX,1
5,NY,1
6,NV,1
7,MA,1
8,IL,1
9,FL,1


In [13]:
### Pandas
clientes_usa = pd.read_sql("""
SELECT State 
FROM customers 
WHERE Country = 'USA' AND State IS NOT NULL
""", connection)
resultado = (clientes_usa['State'].value_counts().reset_index().rename(columns={'index': 'Estado', 'State': 'Cantidad_Clientes'}).sort_values('Cantidad_Clientes', ascending=False))
print(resultado)

   Cantidad_Clientes  count
7                 WI      1
1                 WA      1
10                UT      1
8                 TX      1
2                 NY      1
3                 NV      1
5                 MA      1
6                 IL      1
4                 FL      1
0                 CA      3
9                 AZ      1


### 2 JOINS

##### 2.1 Muestra el nombre de todas las canciones y el género en el que se engloban, lo tengan o no. Llama "Canción" al nombre de la canción y "Género" al nombre del género.

In [14]:
query='''
SELECT 
t.Name AS "Canción",
COALESCE(g.Name, 'Sin género') AS "Género"
FROM 
tracks t
LEFT JOIN 
genres g ON t.GenreId = g.GenreId
ORDER BY 
t.Name;
'''

In [15]:
sql_query(query)

,Canción,Género
0,"""40""",Rock
1,"""?""",TV Shows
2,"""Eine Kleine Nachtmusik"" Serenade In G, K. 525...",Classical
3,#1 Zero,Alternative & Punk
4,#9 Dream,Pop
...,...,...
3498,É que Nessa Encarnação Eu Nasci Manga,Pop
3499,"Étude 1, In C Major - Preludio (Presto) - Liszt",Classical
3500,Óculos,Latin
3501,Óia Eu Aqui De Novo,Soundtrack


##### 2.2 Con la información anterior vamos a repetir el ejercicio 2.1, paso a paso. Primero, carga cada tabla necesaria en 2.1 en un DataFrame, llámalos df_tracks y df_genres.

In [16]:
df_tracks = pd.read_sql("SELECT TrackId, Name, GenreId FROM tracks", connection)
df_genres = pd.read_sql("SELECT GenreId, Name FROM genres", connection)

In [17]:
print("--- DataFrame de canciones (tracks) ---")
print(df_tracks.head())
print("\n--- DataFrame de géneros ---")
print(df_genres.head())

--- DataFrame de canciones (tracks) ---
   TrackId                                     Name  GenreId
0        1  For Those About To Rock (We Salute You)        1
1        2                        Balls to the Wall        1
2        3                          Fast As a Shark        1
3        4                        Restless and Wild        1
4        5                     Princess of the Dawn        1

--- DataFrame de géneros ---
   GenreId                Name
0        1                Rock
1        2                Jazz
2        3               Metal
3        4  Alternative & Punk
4        5       Rock And Roll


In [18]:
df_resultado = pd.merge(
    df_tracks,
    df_genres,
    how='left',
    on='GenreId'
)

In [19]:
df_resultado = (
    df_resultado
    .rename(columns={'Name_x': 'Canción', 'Name_y': 'Género'})
    .fillna({'Género': 'Sin género'})
    [['Canción', 'Género']] 
    .sort_values('Canción') 
)

In [20]:
print(df_resultado)

                                                Canción              Género
3026                                               "40"                Rock
2917                                                "?"            TV Shows
3411  "Eine Kleine Nachtmusik" Serenade In G, K. 525...           Classical
108                                             #1 Zero  Alternative & Punk
3253                                           #9 Dream                 Pop
...                                                 ...                 ...
332               É que Nessa Encarnação Eu Nasci Manga                 Pop
3495    Étude 1, In C Major - Preludio (Presto) - Liszt           Classical
2077                                             Óculos               Latin
1072                                Óia Eu Aqui De Novo          Soundtrack
1076                                Último Pau-De-Arara          Soundtrack

[3503 rows x 2 columns]


##### 2.3 ¿Si colocamos el df_tracks el primero (es decir vamos a hacer df_tracks.merge(df_genres...)), qué valor debe tener el parámetro how? (Ten en cuenta que queremos repetir el mismo JOIN del ejercicio 2.1)

El parámetro how debe ser 'left' para replicar exactamente el mismo JOIN del ejercicio. Así mantendremos todas las canciones de df_tracks incluso si no tienen un género asignado.

##### b. ¿Cuál es la columna de cruce en cada dataframe?¿Usaremos el argumento on, o left_on con right_on?

on='GenreId' (es suficiente y lo más limpio) ya que el nombre de la columna de unión es idéntico en ambos DataFrames

##### 2.4 Haz el join de los dos DataFrames, asignandolo a un tecer DataFrame, llámalo df_join y muéstralo. Observa lo que ocurre en los nombres de las columnas que se llaman igual en los DataFrames originales

In [21]:
df_join = df_tracks.merge(
    df_genres,
    how='left',
    on='GenreId')

In [23]:
print(df_join.head(10))

   TrackId                                   Name_x  GenreId Name_y
0        1  For Those About To Rock (We Salute You)        1   Rock
1        2                        Balls to the Wall        1   Rock
2        3                          Fast As a Shark        1   Rock
3        4                        Restless and Wild        1   Rock
4        5                     Princess of the Dawn        1   Rock
5        6                    Put The Finger On You        1   Rock
6        7                          Let's Get It Up        1   Rock
7        8                         Inject The Venom        1   Rock
8        9                               Snowballed        1   Rock
9       10                               Evil Walks        1   Rock


##### 2.5 Cambia los nombres de las columnas de df_join y quedate solo con las que necesites para repetir el resultado de 2.1

In [27]:
df_resultado = df_join.rename(
    columns={
        'Name_x': 'Canción',
        'Name_y': 'Género'
    }
)[['Canción', 'Género']]

In [28]:
df_resultado['Género'] = df_resultado['Género'].fillna('Sin género')

In [29]:
df_resultado = df_resultado.sort_values('Canción')

In [31]:
print(df_resultado.head(10)) 

                                                Canción              Género
3026                                               "40"                Rock
2917                                                "?"            TV Shows
3411  "Eine Kleine Nachtmusik" Serenade In G, K. 525...           Classical
108                                             #1 Zero  Alternative & Punk
3253                                           #9 Dream                 Pop
601                                     'Round Midnight                Jazz
1832                         (Anesthesia) Pulling Teeth               Metal
569                                       (Da Le) Yaleo                Rock
3044            (I Can't Help) Falling In Love With You              Reggae
3056                                  (Oh) Pretty Woman                Rock


In [32]:
query = '''
SELECT 
COALESCE(g.Name, 'Sin género') AS Género,
COUNT(t.TrackId) AS Número_de_Canciones
FROM 
tracks t
LEFT JOIN 
genres g ON t.GenreId = g.GenreId
GROUP BY 
g.Name
ORDER BY 
2 DESC;  
'''

In [33]:
sql_query(query)

,Género,Número_de_Canciones
0,Rock,1297
1,Latin,579
2,Metal,374
3,Alternative & Punk,332
4,Jazz,130
5,TV Shows,93
6,Blues,81
7,Classical,74
8,Drama,64
9,R&B/Soul,61


In [35]:
conteo_por_genero = (
    df_join.groupby('Name_y', dropna=False)
    .size()
    .reset_index(name='Número_de_Canciones')
    .rename(columns={'Name_y': 'Género'})
    .sort_values('Número_de_Canciones', ascending=False)
)

In [36]:
conteo_por_genero['Género'] = conteo_por_genero['Género'].fillna('Sin género')
print(conteo_por_genero)


                Género  Número_de_Canciones
18                Rock                 1297
12               Latin                  579
13               Metal                  374
1   Alternative & Punk                  332
11                Jazz                  130
23            TV Shows                   93
2                Blues                   81
4            Classical                   74
6                Drama                   64
16            R&B/Soul                   61
17              Reggae                   58
15                 Pop                   48
22          Soundtrack                   43
0          Alternative                   40
10         Hip Hop/Rap                   35
8    Electronica/Dance                   30
24               World                   28
9          Heavy Metal                   28
20    Sci Fi & Fantasy                   26
7       Easy Listening                   24
5               Comedy                   17
3           Bossa Nova          

##### 2.7 Obtén ahora los clientes de Alemania (su nombre completo, ciudad y país) pero sólo los que tengan facturas, con el id y la fecha de esa facturas. Hazlo en SQL

In [37]:
query = '''
SELECT 
c.CustomerId AS ID_Cliente,
c.FirstName || ' ' || c.LastName AS Nombre_Completo,
c.City AS Ciudad,
c.Country AS País,
i.InvoiceId AS ID_Factura,
i.InvoiceDate AS Fecha_Factura
FROM 
customers c
INNER JOIN 
invoices i ON c.CustomerId = i.CustomerId
WHERE 
c.Country = 'Germany'
ORDER BY 
Nombre_Completo, Fecha_Factura DESC;
'''


In [38]:
sql_query(query)

,ID_Cliente,Nombre_Completo,Ciudad,País,ID_Factura,Fecha_Factura
0,37,Fynn Zimmermann,Frankfurt,Germany,367,2013-06-03 00:00:00
1,37,Fynn Zimmermann,Frankfurt,Germany,345,2013-03-01 00:00:00
2,37,Fynn Zimmermann,Frankfurt,Germany,322,2012-11-27 00:00:00
3,37,Fynn Zimmermann,Frankfurt,Germany,193,2011-04-23 00:00:00
4,37,Fynn Zimmermann,Frankfurt,Germany,138,2010-08-23 00:00:00
5,37,Fynn Zimmermann,Frankfurt,Germany,127,2010-07-13 00:00:00
6,37,Fynn Zimmermann,Frankfurt,Germany,6,2009-01-19 00:00:00
7,36,Hannah Schneider,Berlin,Germany,321,2012-11-14 00:00:00
8,36,Hannah Schneider,Berlin,Germany,269,2012-03-26 00:00:00
9,36,Hannah Schneider,Berlin,Germany,247,2011-12-23 00:00:00


##### 2.8 Obtén ahora los clientes brasileños que hicieron compras y sólo estos, y el número de compras que hicieron. Hazlo en SQL

In [39]:
query = '''
SELECT 
c.CustomerId AS ID_Cliente,
c.FirstName || ' ' || c.LastName AS Nombre_Completo,
c.City AS Ciudad,
COUNT(i.InvoiceId) AS Numero_Compras
FROM 
customers c
INNER JOIN 
invoices i ON c.CustomerId = i.CustomerId
WHERE 
c.Country = 'Brazil'
GROUP BY 
c.CustomerId, 
c.FirstName, 
c.LastName, 
c.City
ORDER BY 
Numero_Compras DESC
'''



In [40]:
sql_query(query)

,ID_Cliente,Nombre_Completo,Ciudad,Numero_Compras
0,1,Luís Gonçalves,São José dos Campos,7
1,10,Eduardo Martins,São Paulo,7
2,11,Alexandre Rocha,São Paulo,7
3,12,Roberto Almeida,Rio de Janeiro,7
4,13,Fernanda Ramos,Brasília,7
